In [ ]:
import torch
import matplotlib.pyplot as plt

# Parameters
N = 100          # Number of particles
G = 1.0          # Gravitational constant
dt = 0.01        # Timestep
steps = 200      # Simulation steps

# Initialize on GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
pos = torch.rand(N, 2, device=device) * 10  # Random x, y in [0, 10]
vel = torch.zeros(N, 2, device=device)      # Start at rest
mass = torch.ones(N, device=device)         # Equal masses

# Simulation loop
positions = [pos.cpu().numpy()]  # Store for plotting
for _ in range(steps):
    # Pairwise distances (N x N x 2 tensor)
    r_vec = pos.unsqueeze(1) - pos.unsqueeze(0)  # Shape: (N, N, 2)
    r_sq = (r_vec ** 2).sum(dim=-1) + 1e-6       # Squared distance + softening
    r = r_sq.sqrt()

    # Gravitational force
    force_mag = G * mass.unsqueeze(1) * mass.unsqueeze(0) / r_sq
    force = force_mag.unsqueeze(-1) * (r_vec / r.unsqueeze(-1))  # Direction

    # Net force per particle
    net_force = force.sum(dim=1)  # Sum over all interactions
    accel = net_force / mass.unsqueeze(-1)

    # Update (Euler)
    vel += accel * dt
    pos += vel * dt
    positions.append(pos.cpu().numpy())

# Plot last frame
plt.scatter(positions[-1][:, 0], positions[-1][:, 1], s=10)
plt.title("N-Body Simulation (Final State)")
plt.show()


In [ ]:
import torch
import matplotlib.pyplot as plt
import time

def run_nbody_simulation(N, steps, device):
    """Run N-body simulation on specified device"""
    # Initialize
    pos = torch.rand(N, 2, device=device) * 10
    vel = torch.zeros(N, 2, device=device)
    mass = torch.ones(N, device=device)
    G = 1.0
    dt = 0.01
    
    start_time = time.perf_counter()
    
    # Simulation loop
    for _ in range(steps):
        r_vec = pos.unsqueeze(1) - pos.unsqueeze(0)
        r_sq = (r_vec ** 2).sum(dim=-1) + 1e-6
        r = r_sq.sqrt()
        
        force_mag = G * mass.unsqueeze(1) * mass.unsqueeze(0) / r_sq
        force = force_mag.unsqueeze(-1) * (r_vec / r.unsqueeze(-1))
        
        net_force = force.sum(dim=1)
        accel = net_force / mass.unsqueeze(-1)
        
        vel += accel * dt
        pos += vel * dt
    
    end_time = time.perf_counter()
    return end_time - start_time

# Benchmark parameters
particle_counts = range(500, 3000, 500)
steps = 200
num_runs = 3  # Number of runs to average over

print("\nN-body Simulation Benchmark")
print("-" * 60)
print(f"{'Particles':>10} | {'CPU Time (s)':>15} | {'GPU Time (s)':>15} | {'Speedup':>10}")
print("-" * 60)

for N in particle_counts:
    # CPU benchmark
    cpu_times = []
    for _ in range(num_runs):
        cpu_time = run_nbody_simulation(N, steps, device='cpu')
        cpu_times.append(cpu_time)
    avg_cpu_time = sum(cpu_times) / num_runs
    
    # GPU benchmark (if available)
    if torch.cuda.is_available():
        gpu_times = []
        for _ in range(num_runs):
            torch.cuda.synchronize()  # Ensure GPU operations are completed
            gpu_time = run_nbody_simulation(N, steps, device='cuda')
            torch.cuda.synchronize()
            gpu_times.append(gpu_time)
        avg_gpu_time = sum(gpu_times) / num_runs
        speedup = avg_cpu_time / avg_gpu_time
    else:
        avg_gpu_time = float('nan')
        speedup = float('nan')
    
    print(f"{N:>10} | {avg_cpu_time:>15.3f} | {avg_gpu_time:>15.3f} | {speedup:>10.2f}x")

# Original visualization code
N = 100  # For visualization
device = 'cuda' if torch.cuda.is_available() else 'cpu'
pos = torch.rand(N, 2, device=device) * 10
vel = torch.zeros(N, 2, device=device)
mass = torch.ones(N, device=device)

# Store positions for plotting
positions = [pos.cpu().numpy()]
for _ in range(steps):
    r_vec = pos.unsqueeze(1) - pos.unsqueeze(0)
    r_sq = (r_vec ** 2).sum(dim=-1) + 1e-6
    r = r_sq.sqrt()
    
    force_mag = G * mass.unsqueeze(1) * mass.unsqueeze(0) / r_sq
    force = force_mag.unsqueeze(-1) * (r_vec / r.unsqueeze(-1))
    
    net_force = force.sum(dim=1)
    accel = net_force / mass.unsqueeze(-1)
    
    vel += accel * dt
    pos += vel * dt
    positions.append(pos.cpu().numpy())

# Plot last frame
plt.scatter(positions[-1][:, 0], positions[-1][:, 1], s=10)
plt.title("N-Body Simulation (Final State)")
plt.show()
